## Duckietown Import

In [ ]:
import os 
if not os.path.isdir('gym-duckietown') and not os.path.isdir('../gym-duckietown'):
    branch = "master"
    !git clone --branch {branch} https://github.com/duckietown/gym-duckietown.git
    !pip3 install -e gym-duckietown

In [ ]:
if "/gym-duckietown" not in os.getcwd():
    os.chdir('gym-duckietown')

## Initialize environment

In [ ]:
import gym
import numpy as np
import torch as th
import matplotlib.pyplot as plt
import gym_duckietown
from env import launch_env

from stable_baselines3 import A2C
from stable_baselines3.common.callbacks import CheckpointCallback

In [ ]:
env = launch_env()

In [ ]:
%load_ext tensorboard
tb_log = "..\\log\\" #TODO

In [ ]:
a2c_model = A2C(policy='MlpPolicy',
                env=env,
                learning_rate=5.e-3,
                gamma=0.90,
                tensorboard_log=tb_log,
                verbose=1)

## Learning

In [ ]:
checkpoint_callback = CheckpointCallback(
  save_freq=1000,
  save_path="..\\saves\\", #TODO
  name_prefix="a2c_model",)

In [ ]:
a2c_model.learn(total_timesteps=1000000, callback=checkpoint_callback)

## Model Loading

In [ ]:
env = launch_env()
a2c_model = A2C.load("..\\saves\\rl_model_100000_steps")

## Model Logging

In [ ]:
%tensorboard --logdir tb_log

## Model Testing

In [ ]:
cutoff = 256
with th.no_grad():
    while True:
        obs = env.reset()
        env.render()
        rewards = []
        steps = 0
        while True:
            action, _states = a2c_model.predict(obs, deterministic=True)
            obs, rew, done, misc = env.step(action)
            rewards.append(rew)
            env.render()
            steps += 1
            if done or steps >= cutoff:
                break
        print("mean episode reward:", np.mean(rewards))